# Phase 6: CNN Model — Findings Report

**Summary:** Three CNN versions were trained on 3×64×64 rasterized room images with tabular side-inputs, progressively improving from MAE 17.90 (v1) to 12.40 (v2) to 11.23 (v3). The key finding: the CNN only approached the LightGBM baseline (MAE 11.02) by *weakening* the image branch (bottleneck 256→64) and *strengthening* the tabular branch (skip connection, adding `n_vertices` + `aspect_ratio`). Spatial image information provides negligible predictive value beyond tabular features for this dataset. LightGBM remains the production model for Phase 7 (Grasshopper deployment).

## 1. Architecture Evolution

| Version | Architecture Changes | Test MAE | vs Baseline | W&B Run |
|---------|---------------------|----------|-------------|--------|
| **v1** | Raw concat (256+16+3=275), FC(275→128→1) | 17.90 | +6.88 | [`3wcevehy`](https://wandb.ai/infau/furnisher-surrogate/runs/3wcevehy) |
| **v2** | Image bottleneck 256→64, tabular FC 19→32, head 96→64→1 | 12.40 | +1.38 | [`qutd7leh`](https://wandb.ai/infau/furnisher-surrogate/runs/qutd7leh) |
| **v3** | +`n_vertices`, +`aspect_ratio`, tabular skip connection | 11.23 | +0.21 | [`ld6iz2h4`](https://wandb.ai/infau/furnisher-surrogate/runs/ld6iz2h4) |
| **Baseline** | LightGBM on 14 tabular features | 11.02 | — | [`3t4hiefb`](https://wandb.ai/infau/furnisher-surrogate/runs/3t4hiefb) |

Each version's improvement came from shifting weight toward the tabular branch, not from better image understanding:

- **v1→v2 (−5.50 MAE):** Bottleneck compressed image features from 256→64 dims, tabular FC added (19→32 dims). The model could no longer rely on the noisy image branch and was forced to extract more from tabular features.
- **v2→v3 (−1.17 MAE):** Adding `n_vertices` and `aspect_ratio` as tabular scalars (features the baseline already had) plus a skip connection letting tabular features bypass the image branch entirely.

## 2. Key Finding

The CNN only approached baseline performance when the image branch was *weakened* and the tabular branch was *strengthened*. The spatial image information (room shape and door position rendered as pixels) adds **negligible predictive value** beyond what tabular features already capture.

This is a meaningful negative result: it tells us that for this furnisher algorithm, room score is driven almost entirely by a handful of scalar features (area, vertex count, room type, aspect ratio) rather than by fine-grained spatial layout.

## 3. Per-Room-Type Comparison (v3 vs Baseline)

| Room Type | n (test) | Baseline MAE | CNN v3 MAE | Delta | Better? |
|-----------|----------|-------------|-----------|-------|--------|
| Bedroom | 744 | 10.29 | 9.63 | −0.66 | CNN |
| Living room | 750 | 18.84 | 18.65 | −0.19 | CNN |
| Bathroom | 833 | 3.14 | 3.81 | +0.67 | Baseline |
| WC | 429 | 10.87 | 12.84 | +1.97 | Baseline |
| Kitchen | 833 | 16.89 | 16.67 | −0.22 | CNN |
| Children 1 | 494 | 6.61 | 6.56 | −0.05 | CNN |
| Children 2 | 286 | 7.48 | 8.22 | +0.74 | Baseline |
| Children 3 | 162 | 9.48 | 9.96 | +0.48 | Baseline |
| Children 4 | 63 | 8.97 | 9.62 | +0.65 | Baseline |

CNN v3 beats the baseline on Bedroom, Living room, Kitchen, and Children 1 — these are the room types where spatial layout *might* add value. But the improvements are marginal (< 1 point), while the regressions on WC (+1.97) and Bathroom (+0.67) offset them.

**Overall v3**: MAE=11.23, RMSE=19.18, R²=0.80, Fail/Pass accuracy=0.87.

## 4. Why CNN Doesn't Beat Baseline

Several factors explain why image features fail to add value:

1. **Score is driven by tabular features.** Area (r=+0.37), vertex count, and room type together explain most of the variance. These are all scalar features that tree ensembles handle natively.

2. **Room shape beyond vertex count is weakly predictive.** Whether a room is L-shaped or rectangular matters far less than whether it has 4 or 8 vertices. The image encodes shape detail that the score function doesn't strongly depend on.

3. **3×64×64 resolution may lose critical detail.** Door position and wall proportions at 64px resolution may not provide enough spatial precision for furniture placement reasoning.

4. **Bimodal score distribution favors tree models.** With 28% zeros and 42% scores ≥90, the prediction problem has sharp decision boundaries that gradient-boosted trees handle better than CNNs with MSE loss.

5. **Per-room normalization discards absolute size.** The rasterization normalizes each room to fill the 64×64 canvas, so the image doesn't encode absolute room size — the strongest predictor.

## 5. Potential Improvements (Future Reference)

If spatial features are revisited in future work, these directions could help:

- **Higher resolution** (128×128 or 256×256): May preserve door-wall relationships better
- **Attention mechanisms**: Spatial attention on image features could focus on door/wall interfaces
- **Two-stage model**: Separate zero/nonzero classifier + regression on nonzero rooms
- **Multi-task learning**: Predict room type + score jointly to improve feature extraction
- **Ensemble**: LightGBM + CNN weighted average (per-room-type weights based on validation)
- **Different furnisher algorithms**: The current furnisher may be unusually tabular-friendly; other algorithms might benefit more from spatial features

## 6. Model Artifacts

| Artifact | Path | Contents |
|----------|------|----------|
| CNN v3 checkpoint | `models/cnn_v3.pt` | `model_state_dict`, `config`, normalization stats (`area_mean/std`, `n_verts_mean/std`, `aspect_mean/std`), `epoch`, `val_mae`, `test_mae` |
| CNN v1 checkpoint | `models/cnn_v1.pt` | v1 architecture weights (kept for comparison) |
| Baseline model | `models/baseline_lgbm.joblib` | LightGBM production model |

### W&B Runs

| Run | ID | Link |
|-----|----|------|
| CNN v1 | `3wcevehy` | [wandb.ai](https://wandb.ai/infau/furnisher-surrogate/runs/3wcevehy) |
| CNN v2 | `qutd7leh` | [wandb.ai](https://wandb.ai/infau/furnisher-surrogate/runs/qutd7leh) |
| CNN v3 | `ld6iz2h4` | [wandb.ai](https://wandb.ai/infau/furnisher-surrogate/runs/ld6iz2h4) |
| Baseline | `3t4hiefb` | [wandb.ai](https://wandb.ai/infau/furnisher-surrogate/runs/3t4hiefb) |

## 7. Conclusion

**CNN at best matches LightGBM; spatial image features provide negligible improvement over tabular features.** LightGBM (MAE 11.02) remains the production model for Phase 7 Grasshopper deployment. The CNN work confirms that the furnisher scoring function is dominated by scalar room properties, not fine-grained spatial layout. This is a useful finding that simplifies the deployment architecture — no GPU inference needed in Grasshopper.